In [1]:
# ------------------------------------
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
# ------------------------------------

"""
DESCRIPTION:
    This sample demonstrates how to use agent operations with the 
    Azure AI Search tool from the Azure Agents service using a synchronous client.
    To learn how to set up an Azure AI Search resource,
    visit https://learn.microsoft.com/azure/search/search-get-started-portal

USAGE:
    python sample_agents_azure_ai_search.py

    Before running the sample:

    pip install azure-ai-projects azure-identity

    Set this environment variables with your own values:
    PROJECT_CONNECTION_STRING - the Azure AI Project connection string, as found in your AI Foundry project.
"""

'\nDESCRIPTION:\n    This sample demonstrates how to use agent operations with the \n    Azure AI Search tool from the Azure Agents service using a synchronous client.\n    To learn how to set up an Azure AI Search resource,\n    visit https://learn.microsoft.com/azure/search/search-get-started-portal\n\nUSAGE:\n    python sample_agents_azure_ai_search.py\n\n    Before running the sample:\n\n    pip install azure-ai-projects azure-identity\n\n    Set this environment variables with your own values:\n    PROJECT_CONNECTION_STRING - the Azure AI Project connection string, as found in your AI Foundry project.\n'

In [2]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import AzureAISearchTool

import requests
import sys
import os
import logging
sys.path.append("..")
from utils import load_azd_env, set_azure_ai_project_connection_string, setup_logging, get_file_paths

In [3]:
load_azd_env()
set_azure_ai_project_connection_string()


In [4]:
enbale_logging = False

if enbale_logging:
    setup_logging()

In [5]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

logging.info('create_project_client trigger function started processing a request.')

project_connection_string = os.getenv("AZURE_AI_FOUNDRY_CONNECTION_STRING")

credential = DefaultAzureCredential()

project_client = AIProjectClient.from_connection_string(
    conn_str=project_connection_string,
    credential=credential
    )

In [6]:
# [START create_agent_with_azure_ai_search_tool]
conn_list = project_client.connections.list()
conn_id = ""
for conn in conn_list:
    if conn.connection_type == "CognitiveSearch" and conn.name == "azuresearch2sn":
        conn_id = conn.id
        break

print(conn_id)

In [7]:
# Initialize agent AI search tool and add the search index connection id
ai_search = AzureAISearchTool(index_connection_id=conn_id, index_name="langchain-vector-demo")

In [8]:
# Create agent with AI search tool and process assistant run
with project_client:
    agent = project_client.agents.create_agent(
        model=os.environ["AZURE_AI_DEPLOYMENT_MODEL"],
        name="my-assistant",
        instructions="You are a helpful assistant",
        tools=ai_search.definitions,
        tool_resources=ai_search.resources,
        headers={"x-ms-enable-preview": "true"},
    )
    # [END create_agent_with_azure_ai_search_tool]
    print(f"Created agent, ID: {agent.id}")

    # Create thread for communication
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="What are autonomous agents?",
    )
    print(f"Created message, ID: {message.id}")

    # Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Delete the assistant when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

    # Fetch and log all messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")


HttpResponseError: (None) Invalid tool value(s): azure_ai_search
Code: None
Message: Invalid tool value(s): azure_ai_search